### Advent of Code 2022
#### Day 8: Treetop Tree House

For a rectangular grid of trees of varying heights,

**Part 1.** How many trees are visible (not blocked by same-height or taller trees in their row or column) from outside the patch?

**Part 2.** What's the tree with the best "scenic score" (the product of the number of trees it can see in each cardinal direction)?

For this solution, I didn't use any packages besides the standard distribution and `advent-of-code`.

In [1]:
(require racket
         advent-of-code)

##### Part 1

I built this array as a hashtable with a coordinate struct as the key and the tree height as the value, which makes it easy to check if a particular location falls outside the grid.

In [2]:
(struct posn (r c) #:transparent)

(define (make-tree-grid data)
  (for*/hash ([(row r) (in-indexed data)] [(col c) (in-indexed (in-string row))])
    (values (posn r c) (string->number (string col)))))

(define tree-grid (make-tree-grid (in-lines (open-aoc-input (find-session) 2022 8))))

A tree is visible if there aren't any other trees as tall as it or taller blocking the visibility in at least one of the cardinal directions.  We check in a particular direction by repeatedly stepping outwards from the original coordinate and checking if we've found a blocking tree yet.

The `for/first` returns `#true` if it finds a blocking tree and `#false` if it runs out of trees to check, and the result is negated to turn it into a predicate about whether the tree can see (and be seen from) outside.

In [3]:
(define (can-see-to-outside? p height dr dc h)
  (match-define (posn r c) p)
  (not (for/first ([n (in-naturals 1)]
                   #:do [(define p* (posn (+ r (* n dr)) (+ c (* n dc))))]
                   #:break (not (hash-has-key? h p*))
                   #:when (<= height (hash-ref h p*)))
         #true)))

Now we need a function to check the four cardinal directions and see if at least one of them is unblocked.

In [4]:
(define (visible-in-any-direction? p height h)
  (for*/or ([dr (in-list '(-1 0 1))] [dc (in-list '(-1 0 1))] #:when (= 1 (abs (+ dr dc))))
    (can-see-to-outside? p height dr dc h)))

Finally, add 1 to the count for every tree that's visible:

In [5]:
(define (count-visible-trees trees)
  (for/sum ([(tree-posn height) (in-hash trees)])
           (cond
             [(visible-in-any-direction? tree-posn height trees) 1]
             [else 0])))

(count-visible-trees tree-grid)

1733

##### Part 2

Now we're searching for the tree with the most interior visibility, which is the product of how far we can look in each cardinal direction before running out of visible trees. 

The search is similar, except now we're calculating a score instead of just checking a predicate. We walk one step at a time, increment the counter and check each tree; if we find a blocking tree, we return the counter, and if we run out of trees to check we break and return the previous counter value.  If there aren't any trees in a direction and the `for` body is never evaluated, `for/last` returns `#false`, which is equivalent to a visibility of 0.

In [6]:
(define (scenic-score-in-direction p height dr dc h)
  (match-define (posn r c) p)
  (define score
    (for/last ([n (in-naturals 1)]
               #:do [(define p* (posn (+ r (* n dr)) (+ c (* n dc))))]
               #:break (not (hash-has-key? h p*))
               #:final (<= height (hash-ref h p*)))
      n))
  (if (not score) 0 score))

(define (scenic-score p height h)
  (for*/product ([dr (in-list '(-1 0 1))] [dc (in-list '(-1 0 1))] #:when (= 1 (abs (+ dr dc))))
                (scenic-score-in-direction p height dr dc h)))

With these functions written, calculate each tree's score and find the maximum:

In [7]:
(define (find-best-scenic-score trees)
  (apply max
         (for/list ([(tree-posn height) (in-hash trees)])
           (scenic-score tree-posn height trees))))

(find-best-scenic-score tree-grid)

284648